In [ ]:
!git clone https://github.com/advadj67/hw3_M11221004.git

Cloning into 'hw3_M11221004'...
remote: Enumerating objects: 13628, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 13628 (delta 16), reused 21 (delta 6), pack-reused 13594
Receiving objects: 100% (13628/13628), 1.64 GiB | 36.78 MiB/s, done.
Resolving deltas: 100% (3421/3421), done.
Updating files: 100% (13680/13680), done.


In [ ]:
!git clone https://github.com/3a832069/ML03_dataset.git

Cloning into 'ML03_dataset'...
remote: Enumerating objects: 6807, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6807 (delta 0), reused 3 (delta 0), pack-reused 6804
Receiving objects: 100% (6807/6807), 1.55 GiB | 14.30 MiB/s, done.
Resolving deltas: 100% (3402/3402), done.
Updating files: 100% (6878/6878), done.


In [ ]:
!git clone https://github.com/3a832069/YOLO_pr.git

Cloning into 'YOLO_pr'...
remote: Enumerating objects: 10212, done.
remote: Total 10212 (delta 0), reused 0 (delta 0), pack-reused 10212
Receiving objects: 100% (10212/10212), 1.48 GiB | 39.93 MiB/s, done.
Resolving deltas: 100% (3406/3406), done.
Updating files: 100% (13666/13666), done.


In [2]:
#資料夾準備
import os
# 定義基礎路徑
base_path = '/content/ML03_dataset/貨櫃資料集'
subfolders = ['train', 'test', 'val']

# 創建images和labels資料夾
images_path = os.path.join(base_path, 'images')
labels_path = os.path.join(base_path, 'labels')

os.makedirs(images_path, exist_ok=True)
os.makedirs(labels_path, exist_ok=True)

# 在images和labels資料夾下創建train, test, val子資料夾
for subfolder in subfolders:
    os.makedirs(os.path.join(images_path, subfolder), exist_ok=True)
    os.makedirs(os.path.join(labels_path, subfolder), exist_ok=True)





In [ ]:
!pip install Pillow


In [3]:
from PIL import Image
import os

# 定義路徑
base_path = '/content/ML03_dataset/貨櫃資料集'
input_folders = {
    'train': '訓練集',
    'test': '測試集',
    'val': '驗證集'
}
output_base_path = os.path.join(base_path, 'images')

# 定義函數來調整圖片大小並保存
def resize_and_save(input_folder, output_folder, size=(416, 416)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for img_name in os.listdir(input_folder):
        img_path = os.path.join(input_folder, img_name)
        if os.path.isfile(img_path):
            with Image.open(img_path) as img:
                img = img.resize(size, Image.ANTIALIAS)
                img.save(os.path.join(output_folder, img_name))

# 對每種類型的資料進行處理
for image_type, folder_name in input_folders.items():
    input_folder = os.path.join(base_path, folder_name)
    output_folder = os.path.join(output_base_path, image_type)
    resize_and_save(input_folder, output_folder)

print("所有圖片已成功調整大小並另存。")


<ipython-input-3-a2a289944364>:21: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(size, Image.ANTIALIAS)


所有圖片已成功調整大小並另存。


In [4]:
from PIL import Image
import os

# 定義路徑
original_train_path = '/content/ML03_dataset/貨櫃資料集/測試集'
resized_train_path = '/content/ML03_dataset/貨櫃資料集/images/test'

# 定義函數來打印圖片的shape
def print_image_shapes(folder_path, num_images=2):
    image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    for img_name in image_files[:num_images]:
        img_path = os.path.join(folder_path, img_name)
        with Image.open(img_path) as img:
            print(f"Image: {img_name}, Shape: {img.size}")

# 打印原始圖片的shape
print("Original Images:")
print_image_shapes(original_train_path)

# 打印調整大小後的圖片的shape
print("Resized Images:")
print_image_shapes(resized_train_path)


Original Images:
Image: image_0147.jpg, Shape: (1920, 1080)
Image: image_0751.jpg, Shape: (1280, 720)
Resized Images:
Image: image_0147.jpg, Shape: (416, 416)
Image: image_0751.jpg, Shape: (416, 416)


In [5]:
#標籤處理
import os
import xml.etree.ElementTree as ET

classes = ['container']

# 標籤處理(符合YOLO格式)
def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_xml_to_yolo(xmlPath, output_folder):
    tree = ET.parse(xmlPath)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    # 建立YOLO文件
    txt_filename = os.path.splitext(os.path.basename(xmlPath))[0] + '.txt'
    with open(os.path.join(output_folder, txt_filename), "w") as bbox:

        for obj in root.iter('object'):
            difficult = obj.find('difficult').text
            cls = obj.find('name').text
            if cls not in classes or int(difficult) == 1:
                continue
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text),
                 float(xmlbox.find('xmax').text),
                 float(xmlbox.find('ymin').text),
                 float(xmlbox.find('ymax').text))

            # 轉換成 YOLO 格式
            converted_bb = convert((w, h), b)
            bbox.write(f"{cls_id} {' '.join(str(coord) for coord in converted_bb)}\n")

# 處理整個資料夾
def process_xml_folder(xml_folder, output_folder):
    xml_files = [f for f in os.listdir(xml_folder) if f.endswith('.xml')]
    for xml_file in xml_files:
        xml_path = os.path.join(xml_folder, xml_file)
        convert_xml_to_yolo(xml_path, output_folder)

# 資料夾路徑設置
base_path = '/content/ML03_dataset/貨櫃資料集'
train_xml_folder = os.path.join(base_path, '訓練集_xml')
output_train_folder = os.path.join(base_path, 'labels/train')

val_xml_folder = os.path.join(base_path, '驗證集_xml')
output_val_folder = os.path.join(base_path, 'labels/val')

test_xml_folder = os.path.join(base_path, '測試集_xml')
output_test_folder = os.path.join(base_path, 'labels/test')

# 確認已建立
os.makedirs(output_train_folder, exist_ok=True)
os.makedirs(output_val_folder, exist_ok=True)
os.makedirs(output_test_folder, exist_ok=True)

# 進行格式轉換
process_xml_folder(train_xml_folder, output_train_folder)
process_xml_folder(val_xml_folder, output_val_folder)
process_xml_folder(test_xml_folder, output_test_folder)

print('標籤準備完成')


標籤準備完成


In [6]:
#建立 v5data.yaml
# 定義v5data.yaml文件內容
yaml_content = """# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: /content/ML03_dataset/貨櫃資料集  # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')
test: images/test  # test images (optional)

names:
  0: "container number"
"""

#定義路徑
yaml_file_path = '/content/ML03_dataset/貨櫃資料集/v5data.yaml'

# 寫入文件
with open(yaml_file_path, 'w') as file:
    file.write(yaml_content)



In [7]:
#建立 yolov5.yaml
# 定義yolov5.yaml文件內容
yaml_content = """# Ultralytics YOLO 🚀, AGPL-3.0 license
# YOLOv5 object detection model with P3-P5 outputs. For details see https://docs.ultralytics.com/models/yolov5

# Parameters
nc: 1 # number of classes
names: "container number"

scales: # model compound scaling constants, i.e. 'model=yolov5n.yaml' will call yolov5.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024]
  s: [0.33, 0.50, 1024]
  m: [0.67, 0.75, 1024]
  l: [1.00, 1.00, 1024]
  x: [1.33, 1.25, 1024]

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  - [-1, 1, Conv, [64, 6, 2, 2]] # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]] # 1-P2/4
  - [-1, 3, C3, [128]]
  - [-1, 1, Conv, [256, 3, 2]] # 3-P3/8
  - [-1, 6, C3, [256]]
  - [-1, 1, Conv, [512, 3, 2]] # 5-P4/16
  - [-1, 9, C3, [512]]
  - [-1, 1, Conv, [1024, 3, 2]] # 7-P5/32
  - [-1, 3, C3, [1024]]
  - [-1, 1, SPPF, [1024, 5]] # 9

# YOLOv5 v6.0 head
head:
  - [-1, 1, Conv, [512, 1, 1]]
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 3, C3, [512, False]] # 13

  - [-1, 1, Conv, [256, 1, 1]]
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 3, C3, [256, False]] # 17 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 14], 1, Concat, [1]] # cat head P4
  - [-1, 3, C3, [512, False]] # 20 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 10], 1, Concat, [1]] # cat head P5
  - [-1, 3, C3, [1024, False]] # 23 (P5/32-large)

  - [[17, 20, 23], 1, Detect, [nc]] # Detect(P3, P4, P5)
"""

#定義路徑
yaml_file_path = '/content/ML03_dataset/yolov5.yaml'

# 寫入文件
with open(yaml_file_path, 'w') as file:
    file.write(yaml_content)

In [8]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 kB 12.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [9]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov5n.yaml').load('yolov5n.pt')  # build from YAML and transfer weights


results = model.train(data='/content/ML03_dataset/貨櫃資料集/v5data.yaml', epochs=8, imgsz=416)

model.val()


PRO TIP 💡 Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 5.27M/5.27M [00:00<00:00, 289MB/s]

Transferred 427/427 items from pretrained weights


Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov5n.yaml, data=/content/ML03_dataset/貨櫃資料集/v5data.yaml, epochs=8, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

100%|██████████| 755k/755k [00:00<00:00, 101MB/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:08<00:00,  2.00it/s]


                   all        536        536          1          1      0.995      0.863
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/train
Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv5n summary (fused): 193 layers, 2503139 parameters, 0 gradients, 7.1 GFLOPs


val: Scanning /content/ML03_dataset/貨櫃資料集/labels/val.cache... 536 images, 0 backgrounds, 0 corrupt: 100%|██████████| 536/536 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:06<00:00,  5.45it/s]


                   all        536        536          1          1      0.995      0.864
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f9f072dc280>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [10]:
import shutil

# 指定要下载的文件夹路径
folder_path = "/content/runs"

# 将文件夹压缩成zip文件
shutil.make_archive("/content/runs", 'zip', folder_path)

# 下载压缩文件
from google.colab import files
files.download("/content/runs.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
# 圖像預測+文件內容排序+預測框數量調整(待修改)
from ultralytics import YOLO
import os
import cv2

# 定義來源資料夾和結果儲存資料夾
source_folder = '/content/ML03_dataset/貨櫃資料集/測試集'
output_folder = '/content/pre_results'
prediction_file = '/content/predictions/pred_boxes_for_iou.txt'

# 確保結果儲存資料夾存在
os.makedirs(output_folder, exist_ok=True)
os.makedirs(os.path.dirname(prediction_file), exist_ok=True)

# 讀取所有圖像文件並排序
image_files = [f for f in os.listdir(source_folder) if f.endswith('.jpg') or f.endswith('.png')]
image_files.sort(key=lambda x: int(os.path.splitext(x)[0].split('_')[-1]))

# 計數成功預測和保存的圖片數量
successful_predictions = 0

# 打開預測框資訊文件以寫入
with open(prediction_file, 'w') as txtfile:
    # 遍歷排序後的圖像檔案
    for filename in image_files:
        source_path = os.path.join(source_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # 執行預測
        results = model(source_path)

        # 顯示物件類別和座標
        print(f'Image: {filename}')
        print('Classes:', results[0].boxes.cls)
        print('Coordinates:', results[0].boxes.xyxy)
        print()

        # 保存預測結果的圖像
        # 使用cv2讀取和保存圖像以保留檔案名
        result_img = results[0].plot()  # 繪製預測結果
        cv2.imwrite(output_path, result_img)

        # 計數成功保存的圖片
        successful_predictions += 1

        # 轉換預測框格式並保存到單一的 TXT 文件
        image_id = os.path.splitext(filename)[0]

        # 檢查是否至少有一個預測框存在
        if results[0].boxes:
            box = results[0].boxes[0]
            class_index = int(box.cls.item())
            x_min, y_min, x_max, y_max = box.xyxy.tolist()
            txtfile.write(f"{image_id} {class_index} {x_min} {y_min} {x_max} {y_max}\n")

print('所有圖像預測完成，結果已保存到:', output_folder)
print("預測框已保存到文件中:", prediction_file)

# 驗證預測框文件的行數是否與圖像數量一致
with open(prediction_file, 'r') as txtfile:
    lines = txtfile.readlines()
    print('預測框文件中的行數:', len(lines))
    if len(lines) != len(image_files):
        print(f"警告: 預測框文件中的行數 ({len(lines)}) 與圖像數量 ({len(image_files)}) 不一致。")
    else:
        print("預測框文件中的行數與圖像數量一致。")

# 列出成功保存預測結果的圖片數量
print('成功保存預測結果的圖片數量:', successful_predictions)



image 1/1 /content/ML03_dataset/貨櫃資料集/測試集/image_0001.jpg: 256x416 1 container number, 12.9ms
Speed: 1.3ms preprocess, 12.9ms inference, 1.5ms postprocess per image at shape (1, 3, 256, 416)
Image: image_0001.jpg
Classes: tensor([0.], device='cuda:0')
Coordinates: tensor([[ 779.8394,  104.8817, 1072.2714,  239.9392]], device='cuda:0')



ValueError: not enough values to unpack (expected 4, got 1)

In [ ]:
'''
# 圖像預測+排序調整
from ultralytics import YOLO
import os
import cv2

# 定義來源資料夾和結果儲存資料夾
source_folder = '/content/ML03_dataset/貨櫃資料集/測試集'
output_folder = '/content/pre_results'
prediction_file = '/content/predictions/pred_boxes_for_iou.txt'

# 確保結果儲存資料夾存在
os.makedirs(output_folder, exist_ok=True)
os.makedirs(os.path.dirname(prediction_file), exist_ok=True)

# 讀取所有圖像文件並排序
image_files = [f for f in os.listdir(source_folder) if f.endswith('.jpg') or f.endswith('.png')]
image_files.sort(key=lambda x: int(os.path.splitext(x)[0].split('_')[-1]))

# 打開預測框資訊文件以寫入
with open(prediction_file, 'w') as txtfile:
    # 遍歷排序後的圖像檔案
    for filename in image_files:
        source_path = os.path.join(source_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # 執行預測
        results = model(source_path)

        # 顯示物件類別和座標
        print(f'Image: {filename}')
        print('Classes:', results[0].boxes.cls)
        print('Coordinates:', results[0].boxes.xyxy)
        print()

        # 保存預測結果的圖像
        # 使用cv2讀取和保存圖像以保留檔案名
        result_img = results[0].plot()  # 繪製預測結果
        cv2.imwrite(output_path, result_img)

        # 轉換預測框格式並保存到單一的 TXT 文件
        image_id = os.path.splitext(filename)[0]
        boxes = [{'class_index': box.cls.item(), 'coordinates': [box.xyxy[0], box.xyxy[1], box.xyxy[2], box.xyxy[3]] if len(box.xyxy) == 4 else [box.xyxy[0][0], box.xyxy[0][1], box.xyxy[0][2], box.xyxy[0][3]]} for box in results[0].boxes]

        for box in boxes:
            class_index = box['class_index']
            x_min, y_min, x_max, y_max = box['coordinates']
            txtfile.write(f"{image_id} {class_index} {x_min} {y_min} {x_max} {y_max}\n")

print('所有圖像預測完成，結果已保存到:', output_folder)
print("預測框已保存到文件中:", prediction_file)
'''

In [ ]:
'''
# 圖像預測
from ultralytics import YOLO
import os
import cv2

# 定義來源資料夾和結果儲存資料夾
source_folder = '/content/ML03_dataset/貨櫃資料集/測試集'
output_folder = '/content/pre_results'
prediction_file = '/content/predictions/pred_boxes_for_iou.txt'

# 確保結果儲存資料夾存在
os.makedirs(output_folder, exist_ok=True)
os.makedirs(os.path.dirname(prediction_file), exist_ok=True)


# 打開預測框資訊文件以寫入
with open(prediction_file, 'w') as txtfile:
    # 遍歷資料夾中的所有圖像檔案
    for filename in os.listdir(source_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # 可根據需要添加更多的圖像格式
            source_path = os.path.join(source_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # 執行預測
            results = model(source_path)

            # 顯示物件類別和座標
            print(f'Image: {filename}')
            print('Classes:', results[0].boxes.cls)
            print('Coordinates:', results[0].boxes.xyxy)
            print()

            # 保存預測結果的圖像
            # 使用cv2讀取和保存圖像以保留檔案名
            result_img = results[0].plot()  # 繪製預測結果
            cv2.imwrite(output_path, result_img)

            # 轉換預測框格式並保存到單一的 TXT 文件
            image_id = os.path.splitext(filename)[0]
            boxes = [{'class_index': box.cls.item(), 'coordinates': [box.xyxy[0], box.xyxy[1], box.xyxy[2], box.xyxy[3]] if len(box.xyxy) == 4 else [box.xyxy[0][0], box.xyxy[0][1], box.xyxy[0][2], box.xyxy[0][3]]} for box in results[0].boxes]

            for box in boxes:
                class_index = box['class_index']
                x_min, y_min, x_max, y_max = box['coordinates']
                txtfile.write(f"{image_id} {class_index} {x_min} {y_min} {x_max} {y_max}\n")

print('所有圖像預測完成，結果已保存到:', output_folder)
print("預測框已保存到文件中:", prediction_file)
'''


串流輸出內容已截斷至最後 5000 行。
Image: image_0062.jpg
Classes: tensor([0., 0.], device='cuda:0')
Coordinates: tensor([[ 910.8939,   43.3716, 1044.6105,  144.3883],
        [ 852.9501,   42.4425, 1041.3777,  146.4747]], device='cuda:0')


image 1/1 /content/ML03_dataset/貨櫃資料集/測試集/image_0032.jpg: 256x416 1 container number, 9.9ms
Speed: 1.2ms preprocess, 9.9ms inference, 1.8ms postprocess per image at shape (1, 3, 256, 416)
Image: image_0032.jpg
Classes: tensor([0.], device='cuda:0')
Coordinates: tensor([[ 824.5752,   44.9152, 1070.6616,  159.4689]], device='cuda:0')


image 1/1 /content/ML03_dataset/貨櫃資料集/測試集/image_0291.jpg: 256x416 1 container number, 9.5ms
Speed: 1.0ms preprocess, 9.5ms inference, 1.3ms postprocess per image at shape (1, 3, 256, 416)
Image: image_0291.jpg
Classes: tensor([0.], device='cuda:0')
Coordinates: tensor([[557.5713,   5.7883, 685.9855,  69.4479]], device='cuda:0')


image 1/1 /content/ML03_dataset/貨櫃資料集/測試集/image_0753.jpg: 256x416 1 container number, 9.6ms
Speed: 1.0ms 

In [ ]:
import shutil

# 指定要下载的文件夹路径
folder_path = '/content/pre_results'

# 将文件夹压缩成zip文件
shutil.make_archive("/content/pre_results", 'zip', folder_path)

# 下载压缩文件
from google.colab import files
files.download("/content/pre_results.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
# 真實框載入以及存成txt
import os
from PIL import Image

def get_image_resolution(image_path):
    with Image.open(image_path) as img:
        return img.size

def read_true_boxes(labels_directory, images_directory):
    true_boxes = {}
    for label_file in os.listdir(labels_directory):
        if label_file.endswith('.txt'):
            image_id = os.path.splitext(label_file)[0]
            image_path = os.path.join(images_directory, image_id + ".jpg")  # 假設影像文件格式為.jpg

            # 确保对应的影像文件存在
            if not os.path.exists(image_path):
                print(f"Image file {image_path} does not exist. Skipping.")
                continue

            with open(os.path.join(labels_directory, label_file), 'r') as f:
                lines = f.readlines()
            boxes = []
            image_width, image_height = get_image_resolution(image_path)
            for line in lines:
                parts = line.strip().split()
                class_index = int(parts[0])
                x_center = float(parts[1])
                y_center = float(parts[2])
                width = float(parts[3])
                height = float(parts[4])
                # 計算左上角和右下角坐標
                x1 = int((x_center - width / 2) * image_width)
                y1 = int((y_center - height / 2) * image_height)
                x2 = int((x_center + width / 2) * image_width)
                y2 = int((y_center + height / 2) * image_height)
                boxes.append({'class_index': class_index, 'coordinates': [x1, y1, x2, y2]})
            true_boxes[image_id] = boxes
    return true_boxes

# 資料夾路徑
labels_directory = '/content/ML03_dataset/貨櫃資料集/labels/test'
images_directory = '/content/ML03_dataset/貨櫃資料集/測試集'

# 讀取真實框
true_boxes = read_true_boxes(labels_directory, images_directory)

# 轉換真實框格式
true_boxes_for_iou = [{'image_id': image_id, 'boxes': boxes} for image_id, boxes in true_boxes.items()]

# 按照影像序號排序真實框
true_boxes_for_iou.sort(key=lambda x: int(x['image_id'].split('_')[-1]))

# 將真實框保存到 TXT 文件
output_path = '/content/true_boxes_for_iou.txt'  # 確保路徑正確
with open(output_path, 'w') as txtfile:
    for true_box in true_boxes_for_iou:
        image_id = true_box['image_id']
        for box in true_box['boxes']:
            class_index = box['class_index']
            x_min, y_min, x_max, y_max = box['coordinates']
            txtfile.write(f"{image_id} {class_index} {x_min} {y_min} {x_max} {y_max}\n")

print("真實框轉換完成")


真實框轉換完成


In [16]:
import os

# 定義解析函數來讀取框的座標
def read_boxes_from_txt(file_path):
    boxes = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            image_id = parts[0]
            class_index = int(float(parts[1]))  # 轉換為整數
            x_min = float(parts[2])
            y_min = float(parts[3])
            x_max = float(parts[4])
            y_max = float(parts[5])
            boxes.append({'image_id': image_id, 'class_index': class_index, 'coordinates': [x_min, y_min, x_max, y_max]})
    return boxes

# 定義IoU計算函數
def calculate_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

# 讀取真實框和預測框
true_boxes_file = '/content/true_boxes_for_iou.txt'
pred_boxes_file = '/content/predictions/pred_boxes_for_iou.txt'

true_boxes = read_boxes_from_txt(true_boxes_file)
pred_boxes = read_boxes_from_txt(pred_boxes_file)

# 將框按圖像ID組織起來
true_boxes_by_image = {}
for box in true_boxes:
    image_id = box['image_id']
    if image_id not in true_boxes_by_image:
        true_boxes_by_image[image_id] = []
    true_boxes_by_image[image_id].append(box['coordinates'])

pred_boxes_by_image = {}
for box in pred_boxes:
    image_id = box['image_id']
    if image_id not in pred_boxes_by_image:
        pred_boxes_by_image[image_id] = []
    pred_boxes_by_image[image_id].append(box['coordinates'])

# 計算績效指標
TP = 0
FP = 0
FN = 0
total_iou = 0
total_predictions = 0
iou_threshold = 0.7  # IoU 大於 0.7 才算正樣本

for image_id in true_boxes_by_image:
    if image_id in pred_boxes_by_image:
        true_boxes_for_image = true_boxes_by_image[image_id]
        pred_boxes_for_image = pred_boxes_by_image[image_id]

        matched_true_boxes = set()

        for pred_box in pred_boxes_for_image:
            max_iou = 0
            matched_index = -1
            for true_index, true_box in enumerate(true_boxes_for_image):
                if true_index in matched_true_boxes:
                    continue
                iou = calculate_iou(pred_box, true_box)
                if iou > max_iou:
                    max_iou = iou
                    matched_index = true_index

            total_iou += max_iou
            if max_iou >= iou_threshold:
                TP += 1
                matched_true_boxes.add(matched_index)
            else:
                FP += 1

        FN += len(true_boxes_for_image) - len(matched_true_boxes)
        total_predictions += len(pred_boxes_for_image)
    else:
        # 没有预测框的图像的真是框都算作FN
        FN += len(true_boxes_by_image[image_id])

# 檢查是否有圖像沒有在預測框文件中
for image_id in pred_boxes_by_image:
    if image_id not in true_boxes_by_image:
        total_predictions += len(pred_boxes_by_image[image_id])
        FP += len(pred_boxes_by_image[image_id])

# 計算 Precision、Recall、F1-score、mAP
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
mAP = total_iou / total_predictions if total_predictions > 0 else 0

# 打印所有指標的值
print("TP:", TP)
print("FP:", FP)
print("FN:", FN)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("mAP:", mAP)


TP: 693
FP: 67
FN: 62
Precision: 0.9118421052631579
Recall: 0.9178807947019868
F1 Score: 0.9148514851485148
mAP: 0.7704703454697142


In [ ]:
!pip install pytesseract
!sudo apt-get update
!sudo apt-get install tesseract-ocr
!sudo apt-get install libtesseract-dev
!pip install pytesseract

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,118 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,351 kB in 2s (1,500 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state informati

In [ ]:
import os
from PIL import Image
import cv2
import pytesseract

# 定義解析函數來讀取框的座標
def read_boxes_from_txt(file_path):
    boxes = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            image_id = parts[0]
            class_index = int(float(parts[1]))  # 轉換為整數
            x_min = float(parts[2])
            y_min = float(parts[3])
            x_max = float(parts[4])
            y_max = float(parts[5])
            boxes.append({'image_id': image_id, 'class_index': class_index, 'coordinates': [x_min, y_min, x_max, y_max]})
    return boxes

# 裁切圖片並保存
def load_image(image_path):
    return Image.open(image_path)

def crop_image_with_bbox(image_path, bbox, save_folder):
    image = load_image(image_path)

    # 解析預測框
    x1, y1, x2, y2 = bbox

    # 裁切圖片
    cropped_image = image.crop((x1, y1, x2, y2))

    # 確保保存資料夾存在
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # 取得原始檔名
    base_name = os.path.basename(image_path)

    # 保存裁切後的圖片，使用原始檔名
    cropped_image_path = os.path.join(save_folder, base_name)
    cropped_image.save(cropped_image_path)
    print(f"Cropped image saved at {cropped_image_path}")

    return cropped_image_path

# 將圖片二值化
def binarize_image(image_path, save_folder):
    # 讀取圖片
    image = cv2.imread(image_path)

    # 將圖片轉換為灰度模式
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 使用大津法計算閾值
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # 反轉二值化圖片，將背景設置為白色，文字設置為黑色
    inverted_image = cv2.bitwise_not(binary_image)

    # 確保保存資料夾存在
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # 取得原始檔名
    base_name = os.path.basename(image_path)

    # 組合儲存路徑
    adjusted_image_path = os.path.join(save_folder, base_name)

    # 儲存調整後的圖片
    cv2.imwrite(adjusted_image_path, inverted_image)
    print(f"Adjusted image saved at {adjusted_image_path}")

    return adjusted_image_path

# 文字辨識函式
def image_to_text(image_path):
    # 使用Pillow套件打開圖片
    with Image.open(image_path) as img:
        # 將圖片轉換成文字
        text = pytesseract.image_to_string(img, lang='eng')  # 設定辨識語言，這裡是英文
        return text

# 指定資料夾路徑
image_folder = '/content/ML03_dataset/貨櫃資料集/測試集'
cropped_image_folder = '/content/crop_images'
adjusted_image_folder = '/content/adjust_images'

# 讀取預測框資料
pred_boxes_file = '/content/predictions/pred_boxes_for_iou.txt'
pred_boxes = read_boxes_from_txt(pred_boxes_file)

# 裁切圖片並保存
cropped_image_paths = []
for pred_box in pred_boxes:
    image_file_name = pred_box['image_id'] + ".jpg"  # 假設圖片副檔名是 .jpg，根據實際情況調整
    image_path = os.path.join(image_folder, image_file_name)
    if not os.path.exists(image_path):
        print(f"Image {image_file_name} not found in {image_folder}")
        continue

    # 裁切圖片並保存
    cropped_image_path = crop_image_with_bbox(image_path, pred_box['coordinates'], cropped_image_folder)
    cropped_image_paths.append(cropped_image_path)

# 進行二值化處理
adjusted_image_paths = []
for cropped_image_path in cropped_image_paths:
    adjusted_image_path = binarize_image(cropped_image_path, adjusted_image_folder)
    adjusted_image_paths.append(adjusted_image_path)

# 將文字辨識的結果保存到txt文件中
output_txt_path = '/content/OCR_results.txt'
with open(output_txt_path, 'w', encoding='utf-8') as f:
    for adjusted_image_path in adjusted_image_paths:
        text = image_to_text(adjusted_image_path)
        f.write(f"文字辨識結果 for {os.path.basename(adjusted_image_path)}:\n")
        f.write(text + '\n')
print(f"OCR results saved at {output_txt_path}")


Cropped image saved at /content/crop_images/image_0375.jpg
Cropped image saved at /content/crop_images/image_0629.jpg
Cropped image saved at /content/crop_images/image_0629.jpg
Cropped image saved at /content/crop_images/image_0167.jpg
Cropped image saved at /content/crop_images/image_0608.jpg
Cropped image saved at /content/crop_images/image_0082.jpg
Cropped image saved at /content/crop_images/image_0624.jpg
Cropped image saved at /content/crop_images/image_0670.jpg
Cropped image saved at /content/crop_images/image_0731.jpg
Cropped image saved at /content/crop_images/image_0331.jpg
Cropped image saved at /content/crop_images/image_0179.jpg
Cropped image saved at /content/crop_images/image_0070.jpg
Cropped image saved at /content/crop_images/image_0636.jpg
Cropped image saved at /content/crop_images/image_0610.jpg
Cropped image saved at /content/crop_images/image_0224.jpg
Cropped image saved at /content/crop_images/image_0036.jpg
Cropped image saved at /content/crop_images/image_0643.j

In [ ]:
'''
import os
from PIL import Image
import cv2
import pytesseract

# 定義解析函數來讀取框的座標
def read_boxes_from_txt(file_path):
    boxes = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            image_id = parts[0]
            class_index = int(float(parts[1]))  # 轉換為整數
            x_min = float(parts[2])
            y_min = float(parts[3])
            x_max = float(parts[4])
            y_max = float(parts[5])
            boxes.append({'image_id': image_id, 'class_index': class_index, 'coordinates': [x_min, y_min, x_max, y_max]})
    return boxes

# 裁切圖片並保存
def load_image(image_path):
    return Image.open(image_path)

def crop_and_resize_image_with_bbox(image_path, bbox, save_folder, scale_factor=8):
    image = load_image(image_path)

    # 解析預測框
    x1, y1, x2, y2 = bbox

    # 裁切圖片
    cropped_image = image.crop((x1, y1, x2, y2))

    # 縮放圖片
    new_size = (int(cropped_image.width * scale_factor), int(cropped_image.height * scale_factor))
    resized_image = cropped_image.resize(new_size, Image.ANTIALIAS)

    # 確保保存資料夾存在
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # 取得原始檔名
    base_name = os.path.basename(image_path)

    # 保存裁切後的圖片，使用原始檔名
    cropped_image_path = os.path.join(save_folder, base_name)
    resized_image.save(cropped_image_path)
    print(f"Cropped and resized image saved at {cropped_image_path}")

    return cropped_image_path

# 將圖片二值化
def binarize_image(image_path, save_folder):
    # 讀取圖片
    image = cv2.imread(image_path)

    # 將圖片轉換為灰度模式
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 使用大津法計算閾值
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # 反轉二值化圖片，將背景設置為白色，文字設置為黑色
    inverted_image = cv2.bitwise_not(binary_image)

    # 確保保存資料夾存在
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # 取得原始檔名
    base_name = os.path.basename(image_path)

    # 組合儲存路徑
    adjusted_image_path = os.path.join(save_folder, base_name)

    # 儲存調整後的圖片
    cv2.imwrite(adjusted_image_path, inverted_image)
    print(f"Adjusted image saved at {adjusted_image_path}")

    return adjusted_image_path

# 文字辨識函式
def image_to_text(image_path):
    # 使用Pillow套件打開圖片
    with Image.open(image_path) as img:
        # 將圖片轉換成文字
        text = pytesseract.image_to_string(img, lang='eng')  # 設定辨識語言，這裡是英文
        return text

# 指定資料夾路徑
image_folder = '/content/ML03_dataset/貨櫃資料集/images/test'
cropped_image_folder = '/content/crop_images'
adjusted_image_folder = '/content/adjust_images'

# 讀取預測框資料
pred_boxes_file = '/content/predictions/pred_boxes_for_iou.txt'
pred_boxes = read_boxes_from_txt(pred_boxes_file)

# 裁切圖片並保存
cropped_image_paths = []
for pred_box in pred_boxes:
    image_file_name = pred_box['image_id'] + ".jpg"  # 假設圖片副檔名是 .jpg，根據實際情況調整
    image_path = os.path.join(image_folder, image_file_name)
    if not os.path.exists(image_path):
        print(f"Image {image_file_name} not found in {image_folder}")
        continue

    # 裁切並縮放圖片並保存，設定縮放比例為3
    cropped_image_path = crop_and_resize_image_with_bbox(image_path, pred_box['coordinates'], cropped_image_folder, scale_factor=3)
    cropped_image_paths.append(cropped_image_path)

# 進行二值化處理
adjusted_image_paths = []
for cropped_image_path in cropped_image_paths:
    adjusted_image_path = binarize_image(cropped_image_path, adjusted_image_folder)
    adjusted_image_paths.append(adjusted_image_path)

# 將文字辨識的結果保存到txt文件中
output_txt_path = '/content/OCR_results.txt'
with open(output_txt_path, 'w', encoding='utf-8') as f:
    for adjusted_image_path in adjusted_image_paths:
        text = image_to_text(adjusted_image_path)
        f.write(f"文字辨識結果 for {os.path.basename(adjusted_image_path)}:\n")
        f.write(text + '\n')
print(f"OCR results saved at {output_txt_path}")
'''

<ipython-input-54-947a3b5363f2>:37: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = cropped_image.resize(new_size, Image.ANTIALIAS)


Cropped and resized image saved at /content/crop_images/image_0375.jpg
Cropped and resized image saved at /content/crop_images/image_0629.jpg
Cropped and resized image saved at /content/crop_images/image_0356.jpg
Cropped and resized image saved at /content/crop_images/image_0167.jpg
Cropped and resized image saved at /content/crop_images/image_0608.jpg
Cropped and resized image saved at /content/crop_images/image_0082.jpg
Cropped and resized image saved at /content/crop_images/image_0624.jpg
Cropped and resized image saved at /content/crop_images/image_0670.jpg
Cropped and resized image saved at /content/crop_images/image_0731.jpg
Cropped and resized image saved at /content/crop_images/image_0331.jpg
Cropped and resized image saved at /content/crop_images/image_0179.jpg
Cropped and resized image saved at /content/crop_images/image_0070.jpg
Cropped and resized image saved at /content/crop_images/image_0636.jpg
Cropped and resized image saved at /content/crop_images/image_0610.jpg
Croppe

In [ ]:
'''
import os
from PIL import Image
import cv2
import pytesseract

# 定義解析函數來讀取框的座標
def read_boxes_from_txt(file_path):
    boxes = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            image_id = parts[0]
            class_index = int(float(parts[1]))  # 轉換為整數
            x_min = float(parts[2])
            y_min = float(parts[3])
            x_max = float(parts[4])
            y_max = float(parts[5])
            boxes.append({'image_id': image_id, 'class_index': class_index, 'coordinates': [x_min, y_min, x_max, y_max]})
    return boxes

# 裁切圖片並保存
def load_image(image_path):
    return Image.open(image_path)

def crop_and_resize_image_with_bbox(image_path, bbox, save_folder, scale_factor=8):
    image = load_image(image_path)

    # 解析預測框
    x1, y1, x2, y2 = bbox

    # 裁切圖片
    cropped_image = image.crop((x1, y1, x2, y2))

    # 縮放圖片
    new_size = (int(cropped_image.width * scale_factor), int(cropped_image.height * scale_factor))
    resized_image = cropped_image.resize(new_size, Image.ANTIALIAS)

    # 確保保存資料夾存在
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # 取得原始檔名
    base_name = os.path.basename(image_path)

    # 保存裁切後的圖片，使用原始檔名
    cropped_image_path = os.path.join(save_folder, base_name)
    resized_image.save(cropped_image_path)
    print(f"Cropped and resized image saved at {cropped_image_path}")

    return cropped_image_path

# 將圖片二值化
def binarize_image(image_path, save_folder):
    # 讀取圖片
    image = cv2.imread(image_path)

    # 將圖片轉換為灰度模式
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 使用大津法計算閾值
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # 反轉二值化圖片，將背景設置為白色，文字設置為黑色
    inverted_image = cv2.bitwise_not(binary_image)

    # 確保保存資料夾存在
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # 取得原始檔名
    base_name = os.path.basename(image_path)

    # 組合儲存路徑
    adjusted_image_path = os.path.join(save_folder, base_name)

    # 儲存調整後的圖片
    cv2.imwrite(adjusted_image_path, inverted_image)
    print(f"Adjusted image saved at {adjusted_image_path}")

    return adjusted_image_path

# 文字辨識函式
def image_to_text(image_path):
    # 使用Pillow套件打開圖片
    with Image.open(image_path) as img:
        # 將圖片轉換成文字
        text = pytesseract.image_to_string(img, lang='eng')  # 設定辨識語言，這裡是英文
        return text

# 指定資料夾路徑
image_folder = '/content/ML03_dataset/貨櫃資料集/images/test'
cropped_image_folder = '/content/crop_images'
adjusted_image_folder = '/content/adjust_images'

# 讀取預測框資料
pred_boxes_file = '/content/predictions/pred_boxes_for_iou.txt'
pred_boxes = read_boxes_from_txt(pred_boxes_file)

# 裁切圖片並保存
cropped_image_paths = []
for pred_box in pred_boxes:
    image_file_name = pred_box['image_id'] + ".jpg"  # 假設圖片副檔名是 .jpg，根據實際情況調整
    image_path = os.path.join(image_folder, image_file_name)
    if not os.path.exists(image_path):
        print(f"Image {image_file_name} not found in {image_folder}")
        continue

    # 裁切並縮放圖片並保存，設定縮放比例為3
    cropped_image_path = crop_and_resize_image_with_bbox(image_path, pred_box['coordinates'], cropped_image_folder, scale_factor=3)
    cropped_image_paths.append(cropped_image_path)

# 進行二值化處理
adjusted_image_paths = []
for cropped_image_path in cropped_image_paths:
    adjusted_image_path = binarize_image(cropped_image_path, adjusted_image_folder)
    adjusted_image_paths.append(adjusted_image_path)

# 進行文字辨識，只對指定的圖像進行

target_image_name = 'image_0016'
target_image_path = None
for adjusted_image_path in adjusted_image_paths:
    if target_image_name in os.path.basename(adjusted_image_path):
        target_image_path = adjusted_image_path
        break

if target_image_path:
    text = image_to_text(target_image_path)
    print(f"文字辨識結果 for {os.path.basename(target_image_path)}:")
    print(text)
else:
    print(f"No adjusted image found for {target_image_name}.")


# 進行文字辨識
for adjusted_image_path in adjusted_image_paths:
    text = image_to_text(adjusted_image_path)
    print(f"文字辨識結果 for {os.path.basename(adjusted_image_path)}:")
    print(text)
'''

<ipython-input-52-6282f8686bd5>:37: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = cropped_image.resize(new_size, Image.ANTIALIAS)


Cropped and resized image saved at /content/crop_images/image_0375.jpg
Cropped and resized image saved at /content/crop_images/image_0629.jpg
Cropped and resized image saved at /content/crop_images/image_0356.jpg
Cropped and resized image saved at /content/crop_images/image_0167.jpg
Cropped and resized image saved at /content/crop_images/image_0608.jpg
Cropped and resized image saved at /content/crop_images/image_0082.jpg
Cropped and resized image saved at /content/crop_images/image_0624.jpg
Cropped and resized image saved at /content/crop_images/image_0670.jpg
Cropped and resized image saved at /content/crop_images/image_0731.jpg
Cropped and resized image saved at /content/crop_images/image_0331.jpg
Cropped and resized image saved at /content/crop_images/image_0179.jpg
Cropped and resized image saved at /content/crop_images/image_0070.jpg
Cropped and resized image saved at /content/crop_images/image_0636.jpg
Cropped and resized image saved at /content/crop_images/image_0610.jpg
Croppe

In [ ]:
'''
#嘗試用原圖預測

import os
from PIL import Image
import cv2
import pytesseract

# 定義解析函數來讀取框的座標
def read_boxes_from_txt(file_path):
    boxes = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            image_id = parts[0]
            class_index = int(float(parts[1]))  # 轉換為整數
            x_min = float(parts[2])
            y_min = float(parts[3])
            x_max = float(parts[4])
            y_max = float(parts[5])
            boxes.append({'image_id': image_id, 'class_index': class_index, 'coordinates': [x_min, y_min, x_max, y_max]})
    return boxes

# 裁切圖片並保存
def load_image(image_path):
    return Image.open(image_path)

def crop_image_with_bbox(image_path, bbox, save_folder):
    image = load_image(image_path)

    # 解析預測框
    x1, y1, x2, y2 = bbox

    # 裁切圖片
    cropped_image = image.crop((x1, y1, x2, y2))

    # 確保保存資料夾存在
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # 取得原始檔名
    base_name = os.path.basename(image_path)

    # 保存裁切後的圖片，使用原始檔名
    cropped_image_path = os.path.join(save_folder, base_name)
    cropped_image.save(cropped_image_path)
    print(f"Cropped image saved at {cropped_image_path}")

    return cropped_image_path

# 將圖片二值化
def binarize_image(image_path, save_folder):
    # 讀取圖片
    image = cv2.imread(image_path)

    # 將圖片轉換為灰度模式
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 使用大津法計算閾值
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # 反轉二值化圖片，將背景設置為白色，文字設置為黑色
    inverted_image = cv2.bitwise_not(binary_image)

    # 確保保存資料夾存在
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # 取得原始檔名
    base_name = os.path.basename(image_path)

    # 組合儲存路徑
    adjusted_image_path = os.path.join(save_folder, base_name)

    # 儲存調整後的圖片
    cv2.imwrite(adjusted_image_path, inverted_image)
    print(f"Adjusted image saved at {adjusted_image_path}")

    return adjusted_image_path

# 文字辨識函式
def image_to_text(image_path):
    # 使用Pillow套件打開圖片
    with Image.open(image_path) as img:
        # 將圖片轉換成文字
        text = pytesseract.image_to_string(img, lang='eng')  # 設定辨識語言，這裡是英文
        return text

# 指定資料夾路徑
image_folder = '/content/ML03_dataset/貨櫃資料集/測試集'
cropped_image_folder = '/content/crop_images'
adjusted_image_folder = '/content/adjust_images'

# 讀取預測框資料
pred_boxes_file = '/content/predictions/pred_boxes_for_iou.txt'
pred_boxes = read_boxes_from_txt(pred_boxes_file)

# 裁切圖片並保存
cropped_image_paths = []
for pred_box in pred_boxes:
    image_file_name = pred_box['image_id'] + ".jpg"  # 假設圖片副檔名是 .jpg，根據實際情況調整
    image_path = os.path.join(image_folder, image_file_name)
    if not os.path.exists(image_path):
        print(f"Image {image_file_name} not found in {image_folder}")
        continue

    # 裁切圖片並保存
    cropped_image_path = crop_image_with_bbox(image_path, pred_box['coordinates'], cropped_image_folder)
    cropped_image_paths.append(cropped_image_path)

# 進行二值化處理
adjusted_image_paths = []
for cropped_image_path in cropped_image_paths:
    adjusted_image_path = binarize_image(cropped_image_path, adjusted_image_folder)
    adjusted_image_paths.append(adjusted_image_path)

# 進行文字辨識
for adjusted_image_path in adjusted_image_paths:
    text = image_to_text(adjusted_image_path)
    print(f"文字辨識結果 for {os.path.basename(adjusted_image_path)}:")
    print(text)
'''

Cropped image saved at /content/crop_images/image_0375.jpg
Cropped image saved at /content/crop_images/image_0629.jpg
Cropped image saved at /content/crop_images/image_0356.jpg
Cropped image saved at /content/crop_images/image_0167.jpg
Cropped image saved at /content/crop_images/image_0608.jpg
Cropped image saved at /content/crop_images/image_0082.jpg
Cropped image saved at /content/crop_images/image_0624.jpg
Cropped image saved at /content/crop_images/image_0670.jpg
Cropped image saved at /content/crop_images/image_0731.jpg
Cropped image saved at /content/crop_images/image_0331.jpg
Cropped image saved at /content/crop_images/image_0179.jpg
Cropped image saved at /content/crop_images/image_0070.jpg
Cropped image saved at /content/crop_images/image_0636.jpg
Cropped image saved at /content/crop_images/image_0610.jpg
Cropped image saved at /content/crop_images/image_0224.jpg
Cropped image saved at /content/crop_images/image_0036.jpg
Cropped image saved at /content/crop_images/image_0643.j

In [ ]:
import shutil

# 指定要下载的文件夹路径
folder_path = '/content/adjust_images'

# 将文件夹压缩成zip文件
shutil.make_archive("/content/adjust_images", 'zip', folder_path)

# 下载压缩文件
from google.colab import files
files.download("/content/adjust_images.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil

# 指定要下载的文件夹路径
folder_path = '/content/crop_images'

# 将文件夹压缩成zip文件
shutil.make_archive("/content/crop_images", 'zip', folder_path)

# 下载压缩文件
from google.colab import files
files.download("/content/crop_images.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(1)

1


In [ ]:
#遇到UTF8錯誤時使用
import locale
locale.getpreferredencoding = lambda: "UTF-8"